*Se stai utilizzando Google Colab, esegui le celle sottostanti per inizializzare la macchina.*

In [ ]:
!git clone https://github.com/MatteoH2O1999/iis-alessandrini-lectures.git
!pip install -r ./iis-alessandrini-lectures/requirements.txt
%cd ./iis-alessandrini-lectures/notebooks

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.ticker as mtick
import seaborn as sns

sns.set_theme()
sns.set_context('talk')

# Perché studiare statistica e analisi dei dati?

Cerchiamo di togliere l'ideologia dall'analisi della realtà.

## Delitti violenti

### Chi subisce più spesso crimini di "percosse"? Uomini o donne?

In [ ]:
# Link dataset: http://dati-violenzadonne.istat.it/
# Sezione: Autori e vittime dei delitti denunciati dalle forze di polizia all'autorità giudiziaria: Delitti violenti, sesso, età - reg.

df = pd.read_csv('../datasets/violenze.csv')
df

Prendiamo i dati nazionali

In [ ]:
df = df.loc[df['Territorio'] == 'Italia']
df

Prendiamo solo i dati delle vittime, escludendo i numeri dei denunciati/arrestati per quel delitto

In [ ]:
df = df.loc[df['Tipo dato'] == 'numero di vittime di delitto']
df

E prendiamo infine il numero totale, escludendo l'età.

In [ ]:
df = df.loc[df['Classe di età'] == 'totale']
df

In [ ]:
df = df.sort_values('Sesso', ascending=False)

Plottiamo ora un grafico per avere una visione qualitativa. I diversi anni daranno l'intervallo di confidenza.

In [ ]:
plt.figure(figsize=(16, 9))
sns.barplot(data=df, x='Tipo di delitto', hue='Sesso', y='Value')
plt.show()

Plottiamo ora i grafici per anno delle vittime di percosse

In [ ]:
percosse = df.loc[df['Tipo di delitto'] == 'percosse']
years = list(set(percosse['Seleziona periodo']))
years.sort()
for year in years:
    tmp = percosse.loc[percosse['Seleziona periodo'] == year]
    plt.figure(figsize=(16, 9))
    sns.barplot(data=tmp, x='Tipo di delitto', hue='Sesso', y='Value')
    plt.xlabel("")
    plt.xticks([])
    plt.title(f"Vittime di percosse nel {year}")
    plt.ylim(0, 9000)
    plt.show()

### Chi è più spesso vittima di omicidio? Uomini o donne?

Eseguendo lo stesso processo precedente

In [ ]:
omicidi = df.loc[df['Tipo di delitto'] == 'omicidi volontari consumati']
years = list(set(omicidi['Seleziona periodo']))
years.sort()
for year in years:
    tmp = omicidi.loc[omicidi['Seleziona periodo'] == year]
    plt.figure(figsize=(16, 9))
    sns.barplot(data=tmp, x='Tipo di delitto', hue='Sesso', y='Value')
    plt.xlabel("")
    plt.xticks([])
    plt.title(f"Vittime di omicidio volontario nel {year}")
    plt.ylim(0, 500)
    plt.show()

### Ma allora sto femminismo si aggrappa a nulla?

Non proprio. Andiamo a vedere i dati per stalking e violenze di tipo sessuale

In [ ]:
delitti_sessuali = df.loc[(df['Tipo di delitto'] == 'stalking') | (df['Tipo di delitto'] == 'violenze sessuali')]
years = list(set(delitti_sessuali['Seleziona periodo']))
years.sort()
for year in years:
    tmp = delitti_sessuali.loc[delitti_sessuali['Seleziona periodo'] == year]
    plt.figure(figsize=(16, 9))
    sns.barplot(data=tmp, x='Tipo di delitto', hue='Sesso', y='Value')
    plt.title(f"Vittime di violenze sessuali e stalking nel {year}")
    plt.ylim(0, 13000)
    plt.show()

La situazione direi che è peggiorata molto sull fattore stalking mentre è rimasta pressoché invariata sul fronte violenze.

## Quanto è grande effettivamente il gender pay gap in Italia?

In [ ]:
# Link dataset: http://dati.istat.it/
# Sezione: Lavoro e retribuzioni -> Occupazione dipendente e retribuzioni -> Retribuzioni orarie dei dipendenti del settore privato -> Classe di età

df = pd.read_csv('../datasets/paga_oraria.csv')
df

Rimuoviamo le colonne inutili

In [ ]:
df = df.drop(columns=['Territorio', 'Qualifica contrattuale', 'Ateco 2007', 'Classe di dipendenti', 'Flags'])
df

Selezioniamo soltanto la mediana

In [ ]:
df = df.loc[df['Tipo dato'] == 'Retribuzione lorda  oraria per ora retribuita delle posizioni lavorative dipendenti in euro (mediana).']
df

Proviamo a vedere quanto è ampio il gender pay gap in italia senza considerare le età

Isoliamo i dati che ci serviranno

In [ ]:
total_data = df.loc[df['Classe di età'] == 'totale']
total_data

Ora calcoliamo di anno in anno quanto è esteso il gender pay gap

In [ ]:
years = list(set(total_data['Seleziona periodo']))
years.sort()
gap = []
for year in years:
    tmp = total_data.loc[total_data['Seleziona periodo'] == year]
    gap.append((tmp[tmp.Sesso == 'femmine'].Value.item() / tmp[tmp.Sesso == 'maschi'].Value.item()) * 100)
gap

Proviamo a plottarlo

In [ ]:
plt.figure(figsize=(16, 9))
sns.lineplot(x=years, y=gap)
plt.ylim(0, 101)
plt.title("Percentuale dello stipendio maschile orario medio guadagnato in media da una donna")
plt.xlabel("Anno")
plt.gca().yaxis.set_major_formatter(mtick.PercentFormatter())
plt.show()

Proviamo a restringere l'asse y

In [ ]:
plt.figure(figsize=(16, 9))
sns.lineplot(x=years, y=gap)
plt.ylim(85, 95)
plt.title("Percentuale dello stipendio maschile orario medio guadagnato in media da una donna")
plt.xlabel("Anno")
plt.gca().yaxis.set_major_formatter(mtick.PercentFormatter())
plt.show()

### Quali possono essere le cause di questa disparità?

Iniziamo ristrutturando i dati in modo da avere le percentuali già pronte

In [ ]:
age_classes = set(df['Classe di età'])
age_classes

In [ ]:
years

In [ ]:
new_df = {'Classe di età': [], 'Seleziona periodo': [], 'Gender pay gap in percentuale': []}
for age_class in age_classes:
    for year in years:
        tmp = df.loc[(df['Classe di età'] == age_class) & (df['Seleziona periodo'] == year)]
        male_retr = tmp[tmp.Sesso == 'maschi'].Value.item()
        female_retr = tmp[tmp.Sesso == 'femmine'].Value.item()
        new_df['Classe di età'].append(age_class)
        new_df['Seleziona periodo'].append(year)
        new_df['Gender pay gap in percentuale'].append((female_retr / male_retr) * 100)
new_df = pd.DataFrame.from_dict(new_df)
new_df

Plottiamo il gender pay gap in relazione all'età

In [ ]:
plt.figure(figsize=(16, 9))
sns.lineplot(data=new_df, x='Seleziona periodo', y='Gender pay gap in percentuale', hue='Classe di età')
plt.title("Gender pay gap in Italia diviso per classi di età")
plt.gca().yaxis.set_major_formatter(mtick.PercentFormatter())
plt.show()

Sembra che l'età influisca sulla differenza. Proviamo a plottare come varia il gap con l'età

In [ ]:
df_no_total = new_df.loc[new_df['Classe di età'] != 'totale']
df_no_total

In [ ]:
plt.figure(figsize=(16, 9))
sns.lineplot(data=df_no_total.sort_values('Classe di età'), x='Classe di età', y='Gender pay gap in percentuale', hue='Seleziona periodo')
plt.title("Gender pay gap in Italia diviso per anno")
plt.gca().yaxis.set_major_formatter(mtick.PercentFormatter())
plt.show()